In [1]:
%pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm
import pickle
import numpy as np
from collections import Counter

# Experimenting

In [ ]:
example = pd.read_pickle("./shared/to_vid/datasets/HolyDataset.pkl")
# shape of data
# x_train = HolyDataset[0]
# x_test = HolyDataset[1]
# y_train = HolyDataset[2]
# y_test = HolyDataset[3]
# labels = HolyDataset[4]

In [22]:
df = pd.read_pickle("./Energy_graph/data/processed/REFIT.pkl")

In [26]:
# go over all datasets take 20% of the households rounded down from each dataset and use them for evaluation and use rest for training
# if only one household is in dataset use it for training
# if two,three,four households are in dataset use one for evaluation and rest for training
# labels potentially useless as we will try to predict all appliances
# let's say bellow 20w device is off and above 20w device is on so in kWh with 8s sampling rate we have 0.00004kWh as threshold 4.4e-05
# time_window stand for rows in dataset so all datasets need to have same sampling rate !!!!
# shape the data as an array of tuples of shape (X_train, X_test, Y_train, Y_test) where X_train is an array of shape (time_widow) and Y_train is an array of shape (time_window) and X_test is an array of shape (time_window) and Y_test is an array of shape (time_window)
for file in os.listdir("./Energy_graph/data/training_data/"):
    if not file.endswith(".pkl"):
        continue
    df = pd.read_pickle("./Energy_graph/data/training_data/" + file)
    print(len(list(df.keys())))

6
6
20
5
1
22
5
1
2
1
1
1


In [30]:
data = pd.read_pickle("./Energy_graph/data/training_data/REFIT.pkl")

In [31]:
data["REFIT_1"]

{'aggregate':                      aggregate
 Time                          
 2013-10-09 13:06:16   0.001162
 2013-10-09 13:06:24   0.001169
 2013-10-09 13:06:32   0.001169
 2013-10-09 13:06:40   0.001200
 2013-10-09 13:06:48   0.001200
 ...                        ...
 2015-07-10 11:56:00   0.000400
 2015-07-10 11:56:08   0.000416
 2015-07-10 11:56:16   0.000402
 2015-07-10 11:56:24   0.000413
 2015-07-10 11:56:32   0.000404
 
 [6003014 rows x 1 columns],
 ' fridge':                        fridge
 Time                         
 2013-10-09 13:06:16  0.000164
 2013-10-09 13:06:24  0.000167
 2013-10-09 13:06:32  0.000167
 2013-10-09 13:06:40  0.000164
 2013-10-09 13:06:48  0.000164
 ...                       ...
 2015-07-10 11:56:00  0.000000
 2015-07-10 11:56:08  0.000000
 2015-07-10 11:56:16  0.000000
 2015-07-10 11:56:24  0.000000
 2015-07-10 11:56:32  0.000000
 
 [6003014 rows x 1 columns],
 ' chest freezer':                       chest freezer
 Time                               
 20

In [32]:
df = data["REFIT_1"]["aggregate"]
for device in data["REFIT_1"]:
    if device == "aggregate":
        continue
    df = pd.concat([df, data["REFIT_1"][device]], axis=1)


,aggregate,fridge,chest freezer,upright freezer,tumble dryer,washing machine,dishwasher,computer site,television site,electric heater
Time,,,,,,,,,,
2013-10-09 13:06:16,0.001162,0.000164,0.0000,0.000153,0.0,0.0,0.0,0.0,0.0,0.000002
2013-10-09 13:06:24,0.001169,0.000167,0.0000,0.000153,0.0,0.0,0.0,0.0,0.0,0.000002
2013-10-09 13:06:32,0.001169,0.000167,0.0000,0.000153,0.0,0.0,0.0,0.0,0.0,0.000002
2013-10-09 13:06:40,0.001200,0.000164,0.0000,0.000151,0.0,0.0,0.0,0.0,0.0,0.000002
2013-10-09 13:06:48,0.001200,0.000164,0.0000,0.000151,0.0,0.0,0.0,0.0,0.0,0.000002
...,...,...,...,...,...,...,...,...,...,...
2015-07-10 11:56:00,0.000400,0.000000,0.0001,0.000000,0.0,0.0,0.0,0.0,0.0,0.000002
2015-07-10 11:56:08,0.000416,0.000000,0.0001,0.000000,0.0,0.0,0.0,0.0,0.0,0.000002
2015-07-10 11:56:16,0.000402,0.000000,0.0001,0.000000,0.0,0.0,0.0,0.0,0.0,0.000002


In [34]:

for c in df.columns:
    if c == "aggregate":
        continue

    df[c] = df[c].apply(lambda x: 1 if x > 4.4e-5 else 0)
    

In [62]:
df.iloc[:, 1:]

,fridge,chest freezer,upright freezer,tumble dryer,washing machine,dishwasher,computer site,television site,electric heater
Time,,,,,,,,,
2013-10-09 13:06:16,1,0,1,0,0,0,0,0,0
2013-10-09 13:06:24,1,0,1,0,0,0,0,0,0
2013-10-09 13:06:32,1,0,1,0,0,0,0,0,0
2013-10-09 13:06:40,1,0,1,0,0,0,0,0,0
2013-10-09 13:06:48,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2015-07-10 11:56:00,0,1,0,0,0,0,0,0,0
2015-07-10 11:56:08,0,1,0,0,0,0,0,0,0
2015-07-10 11:56:16,0,1,0,0,0,0,0,0,0


In [59]:
time_window = 2550
windows = []
upper_bound = pd.Timedelta(seconds=32)  
for i in range(0, len(df) - time_window, time_window + 1):
    window = df.iloc[i:i + time_window]
    windows.append(window)






In [60]:

windows[0]
count = 0
test = 0

# this constraint can be relaxed in the future for now we will use it to filter out windows that have gaps of more than 32 seconds for eg. in the future if a window has only 1 gap of one minute we can still use that window

for window in windows:
    test += 1
    time_diffs = window.index.to_series().diff().dropna()
    if  (time_diffs <= upper_bound).all():
        count += 1
    # else:
        # continue
        # print(window.index.to_series().diff().dropna().unique())
    # print(len(window.index.to_series().diff().dropna().unique()))

len(windows) - count


556

# Preprocessing

### Functions

In [18]:
def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]

    # special cases
    if "computer" in string:
        string = "pc"

    if "tv" in string:
        string = "television"

    if "television" in string:
        string = "television"

    if "macbook" in string:
        string = "laptop"
        
    if "ev" == string:
        string = "car charger"
    
    if "toast" in string:
        string = "toaster"
    
    if "modem" in string:
        string = "router"

    # we treat all audio devices as speakers so subwoofer is also a speaker
    if "subwoofer" in string:
        string = "speaker"

    if "speaker" in string:
        string = "speaker"

    if "iron" in string:
        string = "iron"
    
    string = re.sub(' +', ' ', string)

    mapping = pd.read_pickle("./Energy_graph/device_mapping.pkl")
    if string in mapping:
        string = mapping[string]
    
    return string


def process_dictionary(data: dict) -> pd.DataFrame:

    ignored_devices = [
        "light",
        "outlet",
        "sockets",
        "lamp",
        "plug"
    ]
    dfs = []

    for device in data:
        # ignore devices
        if any(ignored_device in device.lower() for ignored_device in ignored_devices):
            continue
        # preprocess device name
        
        device_name = preprocess_string(device)
        df = data[device]
        # rename column to standardized device name
        df.columns = [device_name]

        time_diffs = df.index.to_series().diff()
        median_interval = time_diffs.median()

        # if there is less than a day of data drop the device
        if len(df) < (240 * 60 * 60) / median_interval.total_seconds():
            print("less than a day of data for device: ", device_name)
            continue
        dfs.append(df)

    # concatenate all dataframes
    df = pd.concat(dfs, axis=1)

    # resample to 8s
    df = df.resample("8s").fillna(method="nearest", limit=2)

    # check for gaps in data TODO do this in 

    # drop rows with NaN values
    df.dropna(inplace=True)

    # put 1 if device is on and 0 if device is off
    for c in df.columns:
        if c == "aggregate":
            continue

        df[c] = (df[c] > 4.4e-5).astype(int)

    column_counts = Counter(df.columns)
    duplicates = [col for col, count in column_counts.items() if count > 1]
    # Sum duplicate columns
    for duplicate in duplicates:
        duplicate_cols = [col for i, col in enumerate(df.columns) if col == duplicate]
        df[duplicate] = df[duplicate_cols].sum(axis=1)
        # Drop other duplicate columns if needed
        df = df.loc[:, ~df.columns.duplicated(keep='last')]
    
    
    return df
    


def process_dataset(dataset_path):
    data = pd.read_pickle(dataset_path)
    
    for house in data:
        data[house] = process_dictionary(data[house])
        
    return data








        


### Processing

In [ ]:

# Serial code
path = "./Energy_graph/data/training_data/"
processed_data = {}
for dataset in tqdm(os.listdir(path)):
    if not dataset.endswith(".pkl"):
        continue
    
    data = pd.read_pickle(path + dataset)

    print("processing dataset: ", dataset)
    for house in data:
        data[house] = process_dictionary(data[house])

    processed_data.update(data)
    
    

In [19]:
import concurrent.futures
import multiprocessing

path = "./Energy_graph/data/training_data/"
dataset_paths = [os.path.join(path, dataset) for dataset in os.listdir(path) if dataset.endswith('.pkl')]
    
cpu_count = int(os.cpu_count() / 2)
data_dict = {}

with tqdm(total=len(dataset_paths), desc="Processing datasets", unit="dataset") as progress_bar:
    with concurrent.futures.ProcessPoolExecutor(max_workers=cpu_count) as executor:
        futures = {executor.submit(process_dataset, dataset_path): dataset_path for dataset_path in dataset_paths}
        
        for future in concurrent.futures.as_completed(futures):
            dataset_path = futures[future]
            try:
                processed_data = future.result()
                data_dict.update(processed_data)
            except Exception as e:
                print(f"Dataset {dataset_path} generated an exception: {e}")
            
            progress_bar.update(1)


Processing datasets:   0%|          | 0/11 [00:00<?, ?dataset/s]

less than a day of data for device:  air conditioning
less than a day of data for device:  washing machine
less than a day of data for device:  iron
less than a day of data for device:  television


Processing datasets:   9%|▉         | 1/11 [00:05<00:55,  5.57s/dataset]

less than a day of data for device:  games console
less than a day of data for device:  vacuum cleaner


Processing datasets:  45%|████▌     | 5/11 [00:09<00:07,  1.19s/dataset]

Dataset ./Energy_graph/data/training_data/HES.pkl generated an exception: float() argument must be a string or a real number, not 'pandas._libs.tslibs.offsets.Second'
less than a day of data for device:  iron


Processing datasets:  55%|█████▍    | 6/11 [00:33<00:44,  8.96s/dataset]

less than a day of data for device:  hairdryer straightener


Processing datasets: 100%|██████████| 11/11 [05:51<00:00, 31.96s/dataset]


In [76]:
data_dict.keys()

dict_keys(['REDD_1', 'REDD_2', 'REDD_3', 'REDD_4', 'REDD_5', 'REDD_6', 'HES_1', 'DRED_1', 'DEKN_2', 'DEKN_4', 'DEKN_1', 'DEKN_6', 'DEKN_3', 'HEART_33', 'HEART_7', 'IAWE_1', 'SUST2_1', 'REFIT_13', 'REFIT_6', 'REFIT_1', 'REFIT_21', 'REFIT_8', 'REFIT_9', 'REFIT_20', 'REFIT_7', 'REFIT_15', 'REFIT_12', 'REFIT_4', 'REFIT_3', 'REFIT_18', 'REFIT_11', 'REFIT_16', 'REFIT_17', 'REFIT_10', 'REFIT_19', 'REFIT_2', 'REFIT_5', 'DEDDIAG_8', 'ECO_1', 'ECO_6', 'ECO_2', 'ECO_5', 'ECO_4', 'ECO_3', 'ENERTALK_1', 'ENERTALK_18', 'ENERTALK_12', 'ENERTALK_20', 'ENERTALK_15', 'ENERTALK_6', 'ENERTALK_8', 'ENERTALK_2', 'ENERTALK_11', 'ENERTALK_16', 'ENERTALK_5', 'ENERTALK_7', 'ENERTALK_14', 'ENERTALK_13', 'ENERTALK_19', 'ENERTALK_21', 'ENERTALK_0', 'ENERTALK_4', 'ENERTALK_17', 'ENERTALK_10', 'ENERTALK_3', 'ENERTALK_9', 'UKDALE_5', 'UKDALE_2', 'UKDALE_1', 'UKDALE_3'])

In [14]:
#  save to pickle
with open("./Energy_graph/data/processed_all.pkl", "wb") as f:
    pickle.dump(data_dict, f, pickle.HIGHEST_PROTOCOL)
    


In [4]:
data = pd.read_pickle("./Energy_graph/data/processed_all.pkl")
labels = pd.read_pickle("./Energy_graph/labeles.pkl")
labels


['air conditioning',
 'air exchanger',
 'air exhaust',
 'air handling unit',
 'audio amplifier',
 'audio system',
 'baby monitor',
 'blender',
 'boiler',
 'bouncy castle pump',
 'bread maker',
 'broadband router',
 'car charger',
 'ce appliance',
 'charger',
 'chest freezer ',
 'circulation pump',
 'coffee machine',
 'coffee maker',
 'combination microwave',
 'cooker',
 'dehumidifier/heater',
 'dishwasher',
 'dryer',
 'electric furnace',
 'electric heating element',
 'electric oven',
 'electric space heater',
 'external hard disk',
 'fan',
 'food mixer',
 'food processor',
 'fountain',
 'freezer',
 'fridge freezer ',
 'games console',
 'gaming pc',
 'hair dryer',
 'hairdryer straightener',
 'handmixer',
 'heat basement',
 'heat bedroom #1',
 'heat dining room',
 'heat garage',
 'heat kitchen',
 'heat pump',
 'heater',
 'hi fi',
 'htpc',
 'iron',
 'kettle',
 'kimchi fridge',
 'laptop',
 'microwave',
 'minioven',
 'mobile phone charger',
 'monitor',
 'network attached storage',
 'office 

In [5]:
# to ensure the order of the labels is always the same
labels.sort()

In [80]:
data["REDD_1"]

,aggregate,refrigerator,dishwasher,microwave,electric space heater,electric oven,washer dryer
2011-04-18 09:22:16-04:00,0.000864,0,0,0,0,0,0
2011-04-18 09:22:24-04:00,0.000865,0,0,0,0,0,0
2011-04-18 09:22:32-04:00,0.000867,0,0,0,0,0,0
2011-04-18 09:22:40-04:00,0.000884,0,0,0,0,0,0
2011-04-18 09:22:48-04:00,0.000868,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2011-05-24 15:55:52-04:00,0.000933,1,0,0,0,0,0
2011-05-24 15:56:00-04:00,0.000916,1,0,0,0,0,0
2011-05-24 15:56:08-04:00,0.000912,1,0,0,0,0,0
2011-05-24 15:56:16-04:00,0.000916,1,0,0,0,0,0


In [16]:
time_window = 2550 # this is rows so 2550 * 8s = 5h35m
windows = []
upper_bound = pd.Timedelta(seconds=32)
max_gap = pd.Timedelta(seconds=3600)
X_Y = [] # list of tuples (X, Y)
skip_count_1 = 0
skip_count_2 = 0
total_count = 0

for df in tqdm(data.values()):
    for i in range(0, len(df) - time_window, time_window + 1):
        window = df.iloc[i:i + time_window]
        total_count += 1
        # if there is a gap of more than 30 min or 15 gaps of 32 seconds or more in the window skip it
        time_diffs = window.index.to_series().diff().dropna()
        if  (time_diffs >= max_gap).any():
            skip_count_1 += 1
            continue
        if len(time_diffs[time_diffs > upper_bound]) > 15:
            # print(len(time_diffs[time_diffs > upper_bound]))
            skip_count_2 += 1
            continue
        x = window["aggregate"].values
        devices = [False] * len(labels)
        for c in window.columns:
            if c == "aggregate":
                continue
            on = (window[c] > 0)
            ix = labels.index(c)
            # print(window[c])
            # print(on.any())
            devices[ix] = on.any()

        X_Y.append((x, devices))
        


        windows.append(window)
    # break

# X_Y[9][1].count(False)
print("Total windows: ", total_count, "Skipped windows due to 30min gap: ", skip_count_1, "Skipped windows due to 4 gaps of 32s or more: ", skip_count_2 ,"Procentage skipped: ", (skip_count_1+skip_count_2) / total_count * 100)


100%|██████████| 64/64 [01:16<00:00,  1.19s/it]

Total windows:  50946 Skipped windows due to 30min gap:  658 Skipped windows due to 4 gaps of 32s or more:  211 Procentage skipped:  1.70572763318023


In [86]:
len(labels)

96

In [17]:
# save to pickle
with open("./Energy_graph/data/processed_all_X_Y.pkl", "wb") as f:
    pickle.dump(X_Y, f, pickle.HIGHEST_PROTOCOL)